In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Simulando os dados
np.random.seed(42)
n = 100

# Variáveis independentes
numero_exames = np.random.randint(400, 1000, size=n)
numero_funcionarios = np.random.randint(5, 20, size=n)
dias_uteis = np.random.randint(18, 23, size=n)

# Variável dependente com ruído
consumo_mensal = (2.3 * numero_exames) + (30 * numero_funcionarios) + (50 * dias_uteis) + np.random.normal(0, 1000, size=n)

# DataFrame
df = pd.DataFrame({
    "numero_exames": numero_exames,
    "numero_funcionarios": numero_funcionarios,
    "dias_uteis": dias_uteis,
    "consumo_mensal": consumo_mensal
})

df.head()


**Problema**: Baixa visibilidade no consumo real de insumos nas unidades de diagnóstico.

**Objetivo**: Prever a **quantidade de consumo mensal** com base em variáveis operacionais.

**Variável alvo**: `consumo_mensal`

**Variável preditora (simples)**: `numero_exames`

Para validar uma regressão linear, os seguintes pressupostos devem ser testados:

1. **Linearidade**: Relação linear entre preditores e variável alvo (verificado com scatter plot).
2. **Homoscedasticidade**: Resíduos com variância constante (plot resíduos vs. predição).
3. **Normalidade dos resíduos**: Verificado com histograma e teste de Shapiro-Wilk.
4. **Independência dos resíduos**: Importante em dados temporais (pode ser ignorado aqui).


In [ ]:
X_simple = df[['numero_exames']]
y = df['consumo_mensal']

modelo_simples = LinearRegression()
modelo_simples.fit(X_simple, y)

# Previsões
df['y_pred_simples'] = modelo_simples.predict(X_simple)

# Visualização
plt.figure(figsize=(10, 6))
sns.scatterplot(x='numero_exames', y='consumo_mensal', data=df, label="Dados reais")
sns.lineplot(x='numero_exames', y='y_pred_simples', data=df, color='red', label="Regressão Linear")
plt.title("Regressão Linear Simples: Consumo vs Número de Exames")
plt.xlabel("Número de Exames")
plt.ylabel("Consumo Mensal de Insumos")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
print("Intercepto:", modelo_simples.intercept_)
print("Inclinação:", modelo_simples.coef_[0])


O coeficiente angular (inclinação) indica o quanto o consumo aumenta a cada exame adicional realizado.
O intercepto indica o consumo estimado quando o número de exames é zero.


In [ ]:
r2_simple = r2_score(y, df['y_pred_simples'])
rmse_simple = np.sqrt(mean_squared_error(y, df['y_pred_simples']))

print(f"R²: {r2_simple:.4f}")
print(f"RMSE: {rmse_simple:.2f}")


In [ ]:
X_multi = df[['numero_exames', 'numero_funcionarios', 'dias_uteis']]
modelo_multi = LinearRegression()
modelo_multi.fit(X_multi, y)
df['y_pred_multi'] = modelo_multi.predict(X_multi)


In [ ]:
coeficientes = pd.DataFrame({
    'Variável': X_multi.columns,
    'Coeficiente': modelo_multi.coef_
})
print(coeficientes)


In [ ]:
coeficientes = pd.DataFrame({
    'Variável': X_multi.columns,
    'Coeficiente': modelo_multi.coef_
})
print(coeficientes)


Aqui analisamos quais variáveis têm maior impacto no consumo.

- `numero_exames`: impacto direto
- `numero_funcionarios`: impacto operacional
- `dias_uteis`: influencia no tempo de operação


In [ ]:
residuos = y - df['y_pred_multi']

# Histograma dos resíduos
plt.figure(figsize=(8, 4))
sns.histplot(residuos, kde=True, bins=20)
plt.title("Distribuição dos Resíduos")
plt.xlabel("Resíduo")
plt.show()

# Resíduos vs Preditos
plt.figure(figsize=(8, 4))
sns.scatterplot(x=df['y_pred_multi'], y=residuos)
plt.axhline(0, color='red', linestyle='--')
plt.title("Resíduos vs Valores Preditos")
plt.xlabel("Valor Predito")
plt.ylabel("Resíduo")
plt.show()

# Teste de normalidade dos resíduos
from scipy.stats import shapiro
stat, p_value = shapiro(residuos)
print(f"Shapiro-Wilk p-value: {p_value:.4f}")


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = -cross_val_score(modelo_multi, X_multi, y, scoring='neg_root_mean_squared_error', cv=kf)

print(f"RMSE médio na validação cruzada: {rmse_scores.mean():.2f}")
print(f"RMSEs individuais: {rmse_scores}")


### Aplicação Prática:

Este modelo permite prever o consumo mensal de insumos com base em variáveis operacionais:

- **Benefícios**:
  - Melhor controle de estoque
  - Redução de perdas por falta ou excesso de materiais
  - Apoio ao planejamento logístico e compras

- **Limitações**:
  - Não considera eventos extraordinários
  - Depende da qualidade da entrada de dados
  - Não é adequado para mudanças abruptas sem re-treinamento
